# BUILD A CHILD GRID - OFFLINE NESTING NOTEBOOK VERSION
💡 This notebook allows the user to create a croco chil grid with the offline method:

<div class="alert alert-block alert-info">

To represent fine-scale phenomena, CROCO offers the ability to create zooms with increased resolution. Two options exist 🛠️:

<div style="border: 2px dotted black; padding: 10px; margin: 10px;">
    
##### Offline Zoom 🗺️ ➡️ 🔍

- **Description**: Creation of a grid with the desired resolution, where the boundaries are within a grid of a larger CROCO domain, already modeled.
- **Conditions**: The high-resolution grid takes as initial and boundary conditions those from the larger CROCO simulation. The simulations are independent and no feedback from the high-resolution grid to the coarser grid is possible. The simulations run separately.
- **Advantages**:
  - Independence
  - No constraints on the chosen resolution
- **Disadvantages**:
  - No feedback to the larger domain
  - Zoom boundary forcing depends on the frequency of the parent grid’s history, which may filter out some waves present in the parent grid.
</div>
##### AGRIF Zoom 🔄

- **Description**: AGRIF is a library that allows running CROCO with several embedded domains together. (see documentation in CROCO documentation)
- **Conditions**: Parent and child(s) domains are run simultaneously, coupling the domains at the barotropic time step.
- **Advantages**:
  - Two-way feedback to the parent domain is possible and can significantly improve results of the larger domain
  - Coupling at the highest possible frequency
  - Consistency of numerics
  - Any number of nesting levels is allowed
- **Disadvantages**:
  - Requires simultaneous execution of the parent and child(s) grids
  - Nesting ratio should be 3 or 5
  - Currently, AGRIF zooms in CROCO must have the same vertical resolution as the parent.

</div>

See [online documentation](https://croco-ocean.gitlabpages.inria.fr/croco_pytools/prepro/tuto.zoom.html#offline-zoom-case) for more details 📚

## ✅📋 Dependencies
*Make sure to [build the croco_pyenv environment](https://croco-ocean.gitlabpages.inria.fr/croco_pytools/prepro/tuto.env.html) before using this notebook.*   

In [ ]:
#Allows widget interactivity with user
%matplotlib widget  
%load_ext autoreload
%autoreload 2

# Import standard modules
import numpy as np
import os,sys
    
#----------------------------------------------#
#os.environ['ETS_TOOLKIT'] = 'wx'
sys.path.append("./Modules/")
sys.path.append("./Readers/")
sys.path.append("./Modules/graphicUI_tools/")

#----------------------------------------------#
from tools_grid_inputs import setup_widgets, prepare_variables
#from tools_make_grid import inputs, inputs_smth, EasyGrid, GetMask, GetTopo
from croco_class import CROCO
from tools_create_grids import create_child_grid_offline
from tools_grid_edition import plot_outline_ax, plot_topo_bis

#----------------------------------------------#
#from scipy.spatial import distance
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact_manual, FloatText, VBox
from IPython.display import display    

## ✏️🔧 User's changes
➡️ Launch the next python cell in order to display the widget which allows you to configurate your child grid.  
Then click on the "SAVE" button each time you modify your configuration.  
Concerning the required inputs:  

<table style="width: 100%; text-align: left; border-collapse: collapse;">
  <thead>
    <tr>
      <th style="border: 1px solid #ddd; padding: 8px; background-color: #f4f4f4;">DATA</th>
      <th style="border: 1px solid #ddd; padding: 8px; background-color: #f4f4f4;">SOURCE</th>
      <th style="border: 1px solid #ddd; padding: 8px; background-color: #f4f4f4;">FILE EXTENSION</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;"><strong>⛰️ topo/bathy</strong></td>
      <td style="border: 1px solid #ddd; padding: 8px;">etopo (5,2,1),<br>srtm30, gebco</td>
      <td style="border: 1px solid #ddd; padding: 8px;"><em>NetCDF</em></td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;"><strong>🏖️ coastline</strong></td>
      <td style="border: 1px solid #ddd; padding: 8px;">GSHHS</td>
      <td style="border: 1px solid #ddd; padding: 8px;"><em>shapefile</em></td>
    </tr>
    <tr>
      <td style="border: 1px solid #ddd; padding: 8px;"><strong>🐊 parent CROCO grid</strong></td>
      <td style="border: 1px solid #ddd; padding: 8px;">croco_grd built with make_grid.ipynb</td>
      <td style="border: 1px solid #ddd; padding: 8px;"><em>NetCDF</em></td>
    </tr>
  </tbody>
</table>

In [ ]:
# Call the setup_widgets function to display the widgets and get the function to retrieve saved values
get_saved_config = setup_widgets(parent_grid= True)

## 🧩🌐 Building the child grid with offline method
➡️ By running the next python cell, the configuration saved at the previous step is loaded, and a dictionnary nammed "variables" is extracted from it to run the child-grid-creation-code.
<div class="alert alert-block alert-warning"> 
Tip: if the ouput window is too long 🖱️ right click on the cell annd "Enable scrolling for outputs"
</div>

In [ ]:
# Get the config paramaters defined in the widgets + retain the ones useful for creating the grid
saved_config= get_saved_config()
variables= prepare_variables(saved_config)

#create the grid
chd_grid, inputs, prt= create_child_grid_offline(**variables)

## 👁️🌐 Grid display
Run the next cell to check the child grid before saving it.

In [ ]:
plt.style.use('_mpl-gallery')
figure2, ax2 = plt.subplots(figsize=(8, 8))
figure2.suptitle('Grid outline and bathymetry')
ax2 = plot_outline_ax(prt, ax2)
ax2, cb2 = plot_topo_bis(prt, figure2, ax2)
ax2 = plot_outline_ax(chd_grid, ax2)
ax2, cb2= plot_topo_bis(chd_grid, figure2, ax2)
#The colorbar is set as the one of the parent grid plot min/max
cb2.mappable.set_clim(vmin=prt.h.min(), vmax=prt.h.max())

### 💾 Save grid as necdf
This code creates a Netcdf file containing all grid data (geographic, topo-bathymetric, mask) in the format defined by the classes generated earlier in this notebook. Then it saves it in the ouput repertory.

In [ ]:
CROCO.create_grid_nc(None,saved_config['Files']['output_file'],inputs,chd_grid)